In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [14]:
rate = 0.01
steps = 10000
batch_size = 256

architecture = (28*28, 256, 128, 2, 128, 256, 28*28)
number_layers = len(architecture)
input_size = architecture[0]

In [15]:
X = tf.placeholder(tf.float32, shape=[None, input_size])

weights = []
biases = []

for input_size, output_size in zip(architecture, architecture[1:]):
    weights.append(tf.Variable(tf.random_normal([input_size, output_size])))
    biases.append(tf.Variable(tf.random_normal([output_size])))

def autoencode(x):
    encoding = x
    for W, b in zip(weights, biases):
        encoding = tf.nn.sigmoid(tf.add(tf.matmul(encoding, W), b))
    
    return encoding

In [16]:
loss = tf.reduce_mean(tf.pow(X - autoencode(X), 2))
optimizer = tf.train.RMSPropOptimizer(rate).minimize(loss)

In [20]:
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    
    for i in range(steps):
        batch_x, _ = mnist.train.next_batch(batch_size)
        _, l = sess.run([optimizer, loss], feed_dict={X: batch_x})
        if i % 500 == 0:
            print(f'Loss: {l}')

Loss: 0.4542146921157837
Loss: 0.19566309452056885
Loss: 0.18066604435443878
Loss: 0.16949307918548584
Loss: 0.16512706875801086
Loss: 0.1642482578754425
Loss: 0.1598932296037674
Loss: 0.16096770763397217
Loss: 0.15231773257255554
Loss: 0.15521302819252014
Loss: 0.15037420392036438
Loss: 0.14610178768634796
Loss: 0.14502577483654022
Loss: 0.14276085793972015
Loss: 0.13953350484371185
Loss: 0.13884805142879486
Loss: 0.13386502861976624
Loss: 0.13272221386432648
Loss: 0.13788704574108124
Loss: 0.13481678068637848
